## Import Statements

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import xgboost as xgb
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

## Load data

In [2]:
train_df=pd.read_csv('train.csv')

In [3]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train_df.columns

Index([u'Id', u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea',
       u'Street', u'Alley', u'LotShape', u'LandContour', u'Utilities',
       u'LotConfig', u'LandSlope', u'Neighborhood', u'Condition1',
       u'Condition2', u'BldgType', u'HouseStyle', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'RoofStyle',
       u'RoofMatl', u'Exterior1st', u'Exterior2nd', u'MasVnrType',
       u'MasVnrArea', u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1',
       u'BsmtFinType2', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF',
       u'Heating', u'HeatingQC', u'CentralAir', u'Electrical', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'KitchenQual', u'TotRmsAbvGrd', u'Functional',
       u'Fireplaces', u'FireplaceQu', u'GarageType', u'GarageYrBlt',
       u'GarageFinish',

In [28]:
corrmat = train_df.corr()

In [37]:
corrmat['SalePrice']

Id              -0.021917
MSSubClass      -0.084284
LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
OverallCond     -0.077856
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtFinSF2      -0.011378
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
LowQualFinSF    -0.025606
GrLivArea        0.708624
BsmtFullBath     0.227122
BsmtHalfBath    -0.016844
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr    -0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch   -0.128578
3SsnPorch        0.044584
ScreenPorch      0.111447
PoolArea         0.092404
MiscVal         -0.021190
MoSold           0.046432
YrSold          -0.028923
SalePrice        1.000000
Name: SalePr

In [45]:
core=corrmat['SalePrice'].where(abs(corrmat['SalePrice'])>=0.1)
core=core.dropna().drop('SalePrice')

In [46]:
core

LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
GrLivArea        0.708624
BsmtFullBath     0.227122
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr    -0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch   -0.128578
ScreenPorch      0.111447
Name: SalePrice, dtype: float64

In [47]:
print(type(core))

<class 'pandas.core.series.Series'>


In [52]:
data_df=train_df[core.keys()]

In [53]:
data_df.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch
0,65.0,8450,7,2003,2003,196.0,706,150,856,856,...,1,8,0,2003.0,2,548,0,61,0,0
1,80.0,9600,6,1976,1976,0.0,978,284,1262,1262,...,1,6,1,1976.0,2,460,298,0,0,0
2,68.0,11250,7,2001,2002,162.0,486,434,920,920,...,1,6,1,2001.0,2,608,0,42,0,0
3,60.0,9550,7,1915,1970,0.0,216,540,756,961,...,1,7,1,1998.0,3,642,0,35,272,0
4,84.0,14260,8,2000,2000,350.0,655,490,1145,1145,...,1,9,1,2000.0,3,836,192,84,0,0


In [59]:
data_df['YearBuilt']=2017-data_df['YearBuilt']
data_df['YearRemodAdd']=2017-data_df['YearRemodAdd']

In [60]:
data_df['YearBuilt']

0        14
1        41
2        16
3       102
4        17
5        24
6        13
7        44
8        86
9        78
10       52
11       12
12       55
13       11
14       57
15       88
16       47
17       50
18       13
19       59
20       12
21       87
22       15
23       41
24       49
25       10
26       66
27       10
28       60
29       90
       ... 
1430     12
1431     41
1432     90
1433     17
1434     40
1435     55
1436     46
1437      9
1438     60
1439     38
1440     95
1441     13
1442      9
1443    101
1444     13
1445     51
1446     55
1447     22
1448    107
1449     47
1450     43
1451      9
1452     12
1453     11
1454     13
1455     18
1456     39
1457     76
1458     67
1459     52
Name: YearBuilt, Length: 1460, dtype: int64

In [61]:
data_df['YearRemodAdd']

0       2003
1       1976
2       2002
3       1970
4       2000
5       1995
6       2005
7       1973
8       1950
9       1950
10      1965
11      2006
12      1962
13      2007
14      1960
15      2001
16      1970
17      1967
18      2004
19      1965
20      2006
21      1950
22      2002
23      1976
24      2001
25      2007
26      2000
27      2008
28      1997
29      1950
        ... 
1430    2005
1431    1976
1432    2007
1433    2000
1434    1977
1435    2005
1436    1971
1437    2008
1438    1996
1439    1979
1440    1994
1441    2004
1442    2008
1443    1950
1444    2004
1445    1966
1446    1962
1447    1996
1448    2000
1449    1970
1450    1974
1451    2009
1452    2005
1453    2006
1454    2005
1455    2000
1456    1988
1457    2006
1458    1996
1459    1965
Name: YearRemodAdd, Length: 1460, dtype: int64

In [64]:
data_df.rename({'YearBuilt':'HouseAge','YearRemodAdd':'RemodAge'})

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch
0,65.0,8450,7,14,2003,196.0,706,150,856,856,...,1,8,0,2003.0,2,548,0,61,0,0
1,80.0,9600,6,41,1976,0.0,978,284,1262,1262,...,1,6,1,1976.0,2,460,298,0,0,0
2,68.0,11250,7,16,2002,162.0,486,434,920,920,...,1,6,1,2001.0,2,608,0,42,0,0
3,60.0,9550,7,102,1970,0.0,216,540,756,961,...,1,7,1,1998.0,3,642,0,35,272,0
4,84.0,14260,8,17,2000,350.0,655,490,1145,1145,...,1,9,1,2000.0,3,836,192,84,0,0
5,85.0,14115,5,24,1995,0.0,732,64,796,796,...,1,5,0,1993.0,2,480,40,30,0,0
6,75.0,10084,8,13,2005,186.0,1369,317,1686,1694,...,1,7,1,2004.0,2,636,255,57,0,0
7,NaN,10382,7,44,1973,240.0,859,216,1107,1107,...,1,7,2,1973.0,2,484,235,204,228,0
8,51.0,6120,7,86,1950,0.0,0,952,952,1022,...,2,8,2,1931.0,2,468,90,0,205,0
9,50.0,7420,5,78,1950,0.0,851,140,991,1077,...,2,5,2,1939.0,1,205,0,4,0,0
